In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [16]:
train_data = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv')
test_data = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv')
sample_sol = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv')

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [5]:
def new_col(df):
  df[['flight_country', 'flight_number']]=df["flight"].str.split("-", expand=True)
  df.drop(['id', 'flight'], axis=1, inplace=True)

new_col(train_data)
new_col(test_data)

In [6]:
y = train_data['price']
X = train_data.drop('price', axis=1)

In [7]:
cat_var = ['airline', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'flight_country']
num_var = ['duration', 'days_left', 'flight_number']
def my_pipeline(df):
  df = pd.get_dummies(df, columns=cat_var)
  df[num_var] = StandardScaler().fit_transform(df[num_var])
  return df

X = my_pipeline(X)
test_data = my_pipeline(test_data)

In [9]:
rfr_p = []
# lr_p = []
# dtr_p = []
xgb_p = []

kf = KFold(n_splits=10, shuffle=True)
for k, (train, test) in enumerate(kf.split(X, y)):
  rfr = RandomForestRegressor()
  rfr.fit(X.iloc[train], y.iloc[train])
  rfr_p.append(rfr.predict(X.iloc[test]))

#   lr = LinearRegression()
#   lr.fit(X.iloc[train], y.iloc[train])
#   lr_p.append(lr.predict(X.iloc[test]))

#   dtr = DecisionTreeRegressor()
#   dtr.fit(X.iloc[train], y.iloc[train])
#   dtr_p.append(dtr.predict(X.iloc[test]))

  xgb = XGBRegressor()
  xgb.fit(X.iloc[train], y.iloc[train])
  xgb_p.append(xgb.predict(X.iloc[test]))

  print(k, np.sqrt(mean_squared_error(rfr_p[k], y[test])),
#         np.sqrt(mean_squared_error(lr_p[k], y[test])),
#         np.sqrt(mean_squared_error(dtr_p[k], y[test])),
        np.sqrt(mean_squared_error(xgb_p[k], y[test]))
        )

0 3245.611390881165 7046.218471621936 4114.475328094702 3520.065266753173
1 3335.846205969087 6436.150789136314 4179.004579203042 3416.287082431908
2 3112.731565408391 6561.526198436873 4153.019926631703 3385.1486841999576
3 3182.4651722229332 6679.967870918243 4104.650956963333 3256.573377287967
4 3510.981193841923 7233.33665850778 4587.267632071405 3679.362702915575
5 2856.7275865437327 6447.573219902198 4168.157906017957 3147.3895382088563
6 3348.010626593305 6671.77344388677 4564.185421846926 3423.882856826602
7 3434.826481628416 6477.41285024353 4514.608444483309 3486.6029994920855
8 3302.3595015450815 6956.401941176425 4109.2926718749295 3288.0472179712056
9 3260.7852817147655 6636.06946513206 4137.118188500904 3341.6881907922416


In [17]:
prediction = rfr.predict(test_data)
sample_sol['price'] = prediction
sample_sol.head()

,id,price
0,1,50771.75
1,2,62702.38
2,3,23068.76
3,4,2651.46
4,5,5739.99


In [18]:
sample_sol.to_csv('submission.csv')